In [9]:
%pwd

'c:\\Users\\abhil\\OneDrive\\Desktop\\ABHIL\\Demo\\Gadget_recommendation_Engine_Using_LLM'

In [8]:
import os

os.chdir('..')

In [11]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter 

In [12]:
#extract text from PDF file
def load_pdf_file(data):
    loader = DirectoryLoader(data, 
                             glob="*.pdf", 
                             loader_cls=PyPDFLoader)
    
    documents = loader.load()
    
    return documents

In [13]:
extracted_data = load_pdf_file(data='Data/')

In [15]:
#extracted_data

In [16]:
#split text into smaller chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    texts_chunks = text_splitter.split_documents(extracted_data)

    return texts_chunks

In [17]:
text_chunks = text_split(extracted_data)
print(f"length of text chunks: {len(text_chunks)}")

length of text chunks: 5859


In [23]:
from langchain.embeddings import HuggingFaceEmbeddings

In [24]:
def download_hugging_face_embeddings():

    # Download the embedding model
    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    return embedding_model

In [29]:
embeddings = download_hugging_face_embeddings()

c:\Users\abhil\miniconda3\envs\gadgetbot\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\abhil\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling 

In [36]:
query_results = embeddings.embed_documents("Hello, how are you?")
print("length", len(query_results))

length 19


In [37]:
#query_results

In [39]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
Pinecone_API_KEY = os.environ.get("PINECONE_API_KEY")
Pinecone_Environment = os.environ.get("PINECONE_ENVIRONMENT")

In [42]:
from pinecone.grpc import PineconeGRPC as pinecone
from pinecone import ServerlessSpec
import os

pc = pinecone(api_key=Pinecone_API_KEY)

index_name = "gadgetbot"


pc.create_index(
    name=index_name,
    dimension=384,  # Dimension of the embedding model
    metric="cosine",  # Similarity metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

{
    "name": "gadgetbot",
    "metric": "cosine",
    "host": "gadgetbot-hk9iesg.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [44]:
import os
os.environ["PINECONE_API_KEY"] = Pinecone_API_KEY

In [46]:
from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings to Pinecone
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name = index_name,
    embedding=embeddings,
)

In [47]:
#Load existing index if it already exists
# Uncomment the following lines if you want to load an existing index instead of creating a new one

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings to Pinecone
docsearch = PineconeVectorStore.from_existing_index(
    index_name = index_name,
    embedding=embeddings,
)

In [48]:
docsearch